In [57]:
import importlib
importlib.reload(chain)

<module 'chain' from '/Volumes/Repository/Projects/ffa/chain.py'>

In [ ]:
import dash
from dash import Dash, dcc, html, Input, Output, callback
import dash_bootstrap_components as dbc
import chain

app = Dash(external_stylesheets=[dbc.themes.FLATLY],  
           suppress_callback_exceptions=True,         
           prevent_initial_callbacks='initial_duplicate')

main_menu = dbc.Nav(
    [
        dbc.NavItem(dbc.NavLink("市场全景", active=True, href="/"), ),
        dbc.DropdownMenu(
            [
                dbc.DropdownMenuItem("钢铁产业链", href="/chain/black_metals"), 
                dbc.DropdownMenuItem("贵金属产业链", href="/chain/precious_metals"),
                dbc.DropdownMenuItem("铝产业链", href="/chain/alum"),
                dbc.DropdownMenuItem("铜产业链", href="/chain/copper"),
             ],
            label="金属",
            nav=True,
        ),
        dbc.DropdownMenu(
            [
                dbc.DropdownMenuItem("动力煤产业链", href="/chain/coal"), 
                dbc.DropdownMenuItem("石油产业链", href="/chain/oil"),
                dbc.DropdownMenuItem("原油产业链", href="/chain/crude"),
             ],
            label="能源",
            nav=True,
        ),
        dbc.DropdownMenu(
            [
                dbc.DropdownMenuItem("化工产业链", href="/chain/chemical"),
                dbc.DropdownMenuItem("沥青产业链", href="/chain/asphalt"),
                dbc.DropdownMenuItem("橡胶产业链", href="/chain/rubber"),
             ],
            label="化工",
            nav=True,
        ),        
        dbc.DropdownMenu(
            [
                dbc.DropdownMenuItem("菜籽油产业链", href="/chain/rapeseed_oil"), 
                dbc.DropdownMenuItem("大豆产业链", href="/chain/soybeans"),
                dbc.DropdownMenuItem("糖产业链", href="/chain/sugar"),
                dbc.DropdownMenuItem("小麦产业链", href="/chain/wheat"),
                dbc.DropdownMenuItem("玉米产业链", href="/chain/corn"),
                dbc.DropdownMenuItem("棕榈油产业链", href="/chain/palm_oil"),
                dbc.DropdownMenuItem("生猪产业链", href="/chain/pig"),
             ],
            label="农产品",
            nav=True,
        ),
        dbc.NavItem(dbc.NavLink("自选", active=True, href="#"), ),
        dbc.DropdownMenu(
            [
                dbc.DropdownMenuItem("数据管理", href="/data_manage"),
                dbc.DropdownMenuItem("品种管理", href="/variety_manage"),
                dbc.DropdownMenuItem("设置", href="/setting"),
             ],
            label="我的",
            nav=True,
        ),                
    ],
    navbar=True,
    # horizontal='end'
)

search_bar = dbc.Row(
    [
        dbc.Col(dbc.Input(type="search", placeholder="Search")),
        dbc.Col(
            dbc.Button(
                "Search", color="primary", className="ms-2", n_clicks=0
            ),
            width="auto",
        ),
    ],
    className="g-0 ms-auto flex-nowrap mt-3 mt-md-0",
    align="center",
)

nav_bar = dbc.Navbar(
    dbc.Container(
        [
            html.A(
                # Use row and col to control vertical alignment of logo / brand
                dbc.Row(
                    [
                        dbc.Col(html.Img(src='./logo.png', height="30px")),
                        dbc.Col(dbc.NavbarBrand("Futures Nexus", className="ms-2")),
                    ],
                    align="start",
                    className="g-0",
                ),
                href="/",
                style={"textDecoration": "none"},
            ),
            main_menu,
            dbc.NavbarToggler(id="navbar-toggler", n_clicks=0),
            dbc.Collapse(
                search_bar,
                id="navbar-collapse",
                is_open=False,
                navbar=True,
            ),            
        ],
    ),
    # color = 'dark',
    # dark = True,
)

# the styles for the main content position it to the right of the sidebar and
# add some padding.
CONTENT_STYLE = {
    "margin-left": "1rem",
    "margin-right": "1rem",
    # "padding": "2rem 1rem",
}

content = html.Div(id="page-content", style=CONTENT_STYLE)

app.layout = dbc.Container(
    [
        dcc.Location(id='url', refresh=False),
        html.Div([
            dbc.Row([         
                nav_bar,
                html.Hr(),
            ],
            align='start'),    
            dbc.Row([
                content,
            ],
            align='start'),    
        ])         

    ],
    className="p-5",
    fluid=True,
)

page_not_found = html.Div(
    [
        html.H1("404: Not found", className="text-danger"),
        html.Hr(),
        html.P(f"The page was not recognised..."),
    ],
    className="p-3 bg-light rounded-3",
)

home_page = html.Div(
    html.P("This is the content of the home page!")
)

chain.callback(app)

@app.callback(Output("page-content", "children"), [Input("url", "pathname")])
def render_page_content(pathname):
    print('Main Router: ', pathname)
    path_parts = pathname.strip('/').split('/')    
    module = path_parts[0]
    chain_name = path_parts[1] if len(path_parts) > 1 else None
    variety = path_parts[2] if len(path_parts) > 2 else None
    analysis_type = path_parts[3] if len(path_parts) > 3 else None

    if pathname == '/' or pathname =='':
        page_content = home_page
    elif module == 'chain': 
        print(chain_name, variety, analysis_type)
        chain_page = chain.page_router(chain_name, variety, analysis_type)
        page_content = page_not_found if chain_page is None else chain_page

    return page_content

if __name__ == "__main__":
    app.run_server(debug=True, port=9999)